In [1]:
import json 
with open('./data/question.json', 'r', encoding='utf-8') as f1:
    data1_raw = json.load(f1)

data1 = []
for item in data1_raw:
    for qa in item['data']:
        data1.append({
            'question':qa['question'],
            'answer': qa['answer']
        })


In [2]:
with open('./data/QA_output_multihop_allQuyetdinh.json', 'r', encoding='utf-8') as f2:
    data2_raw = json.load(f2)

data2 = []
data2 = [{'question': item['question'], 'answer': item['answer']} for item in data2_raw]


In [3]:
with open('./data/QA_output_multihop_allThongtu.json', 'r', encoding='utf-8') as f3:
    data3_raw = json.load(f3)

data3 = []
data3 = [{'question': item['question'], 'answer': item['answer']} for item in data3_raw]
print(data3[0])

{'question': 'Một tổ chức sự nghiệp công lập trực thuộc Bộ Nông nghiệp và Môi trường, chuyên nghiên cứu về giống cây trồng mới, muốn đề xuất một nhiệm vụ KHCN cấp bộ để cải tạo một giống lúa địa phương. Tuy nhiên, tổ chức này đang chậm nộp báo cáo kết quả ứng dụng của một đề tài KHCN cấp bộ đã nghiệm thu trước đó 7 tháng. Tổ chức này có đủ điều kiện để đề xuất nhiệm vụ KHCN cấp bộ mới không, và nếu không thì quy định nào sẽ được áp dụng?', 'answer': 'Tổ chức này không đủ điều kiện đề xuất nhiệm vụ KHCN cấp bộ mới. Theo Điều 8 Thông tư 2025/TT-BNNMT, tổ chức, cá nhân đề xuất đặt hàng nhiệm vụ KHCN thông qua hệ thống quản lý KHCN của Bộ. Tuy nhiên, theo điểm a khoản 2 Điều 10 Thông tư này, tổ chức chưa hoàn thành việc báo cáo ứng dụng kết quả nhiệm vụ KHCN sử dụng ngân sách nhà nước sẽ không đủ điều kiện đăng ký tham gia tuyển chọn, giao trực tiếp chủ trì nhiệm vụ. Vì tổ chức này chậm nộp báo cáo ứng dụng kết quả nhiệm vụ KHCN đã nghiệm thu trước đó, nên không đủ điều kiện để đề xuất nhi

In [4]:
with open("./data/merged_questions.json", 'r', encoding='utf-8') as f4:
    data_raw4 = json.load(f4)

data4 = []
for item in data_raw4:
    data4.append({
        "question": item["question"],
        "answer": item["answer"]
    })
print(data4[0])

{'question': 'Tôi là chủ nhiệm một đề tài cấp Bộ, vậy mức thù lao tối đa một tháng của tôi được nhận là bao nhiêu?', 'answer': 'Theo Điều 4, khoản 2 của Quy định, mức thù lao tối đa cho chủ nhiệm nhiệm vụ khoa học và công nghệ cấp Bộ là không quá 30.000.000 đồng/người/tháng.'}


In [5]:
merge_data = data1 + data2 + data3 + data4
print(len(merge_data))

444


In [6]:
with open('./output/data.json', 'w', encoding='utf-8') as fout:
    json.dump(merge_data, fout, ensure_ascii=False, indent=2)

print(f"✅ Gộp xong. Tổng số QA: {len(merge_data)}")

✅ Gộp xong. Tổng số QA: 444


In [7]:
import json

# Load dữ liệu đã merge
with open("./output/data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Format lại thành dạng chat
formatted = []
for item in data:
    chat_item = {
        "messages": [
            {"role": "user", "content": item["question"]},
            {"role": "assistant", "content": item["answer"]}
        ]
    }
    formatted.append(chat_item)

# Lưu ra file JSONL cho fine-tuning
with open("./output/finetune_data.jsonl", "w", encoding="utf-8") as f:
    for line in formatted:
        f.write(json.dumps(line, ensure_ascii=False) + "\n")

print(f"✅ Đã chuẩn bị xong {len(formatted)} mẫu dữ liệu.")


✅ Đã chuẩn bị xong 444 mẫu dữ liệu.


In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
import torch
from transformers import BitsAndBytesConfig

c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
model_name = "Qwen/Qwen3-4B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [10]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",         
    bnb_4bit_use_double_quant=True,     # Dùng double quantization để giảm lỗi
    bnb_4bit_compute_dtype=torch.float16  
)

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    quantization_config=bnb_config,
    device_map="auto", 
    use_cache=True
)

The 8-bit optimizer is not available on your device, only available on CUDA for now.
Loading checkpoint shards: 100%|██████████| 3/3 [00:49<00:00, 16.44s/it]


In [11]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1
)

model = get_peft_model(model, peft_config)

In [13]:
dataset = load_dataset("json", data_files="./output/finetune_data.jsonl", split="train")


Generating train split: 444 examples [00:00, 40428.78 examples/s]


In [14]:
def tokenize(example):
    messages = example["messages"]
    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", truncation=True, max_length=1024).squeeze()
    return {"input_ids": input_ids, "labels": input_ids}

tokenized_dataset = dataset.map(tokenize, remove_columns=["messages"])


Map: 100%|██████████| 444/444 [00:00<00:00, 895.92 examples/s] 


In [15]:
# Training config
training_args = TrainingArguments(
    output_dir="./vistral-law-finetuned",
    per_device_train_batch_size=1,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=500,
    save_total_limit=1,
    fp16=True,
    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    data_collator=data_collator,
)

trainer.train()

c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,3.849800
20,2.545200
30,3.200100
40,2.668800
50,2.726900
60,2.910200
70,2.779100
80,2.077400
90,1.926500
100,2.056600


c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=1332, training_loss=1.3271178132420904, metrics={'train_runtime': 13397.2155, 'train_samples_per_second': 0.099, 'train_steps_per_second': 0.099, 'total_flos': 6267544725215232.0, 'train_loss': 1.3271178132420904, 'epoch': 3.0})

In [ ]:
# from huggingface_hub import upload_folder

# trainer.save_model()

# model = model.merge_and_unload()  # Merge adapter vào base
# merged_path = "./output/qwen-law-finetuned-merged"
# model.save_pretrained(merged_path)
# tokenizer.save_pretrained(merged_path)

# # Upload thư mục merged lên Hugging Face Hub
# upload_folder(
     
#     folder_path=merged_path,
#     path_in_repo="."
# )

c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\peft\tuners\lora\bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


NameError: name 'upload_folder' is not defined